# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import time
import dateutil.parser
import requests
from typing import List, Dict

%matplotlib inline
plt.rcParams['figure.figsize'] = [17, 7]

# Get Order Book Data

In [33]:
def orderbook_data(depth: int, contract: str = "ETHUSD", raw_data=False):

    url = "https://www.bitmex.com/api/v1/orderBook/L2"
    book_data = {'buys': 0, 'sells': 0}
    raw_book_data = {'total_buys': 0, 'sum_of_buys': 0, 'furthest_5_buy_prices': [],
                    'total_sells': 0, 'sum_of_sells': 0, 'furthest_5_sell_prices': []}


    # Set inputs
    data = dict()
    data['symbol'] = contract
    data['depth'] = depth

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        response_data = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if response_data is not None:
        if raw_data:
            for idx, data in enumerate(response_data):

                if idx < 5:
                    raw_book_data['furthest_5_buy_prices'].append(data['price'])

                if idx > len(response_data)-6:
                    raw_book_data['furthest_5_sell_prices'].append(data['price'])

                if data['side'] == "Buy":
                    raw_book_data["sum_of_buys"] += data["size"]
                    raw_book_data['total_buys'] += 1

                elif data['side'] == "Sell":
                    raw_book_data["sum_of_sells"] += data["size"]
                    raw_book_data['total_sells'] += 1

            return raw_book_data

        else:
            for data in response_data:
                if data['side'] == "Buy":
                    book_data["buys"] += data["size"]

                elif data['side'] == "Sell":
                    book_data["sells"] += data["size"]
    
            return book_data

# Get current price

In [23]:
def get_curr_price(contract: str) -> str:

    url = "https://www.bitmex.com/api/v1/trade/bucketed"

    data = dict()
    data['symbol'] = contract
    data['partial'] = True  # returns a candle if it is not finished yet
    data['binSize'] = '1m'
    data['count'] = 1   # how many candles we can return (500 max)
    data['reverse'] = True

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        raw_candles = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if raw_candles is not None:
        return raw_candles[0]['close']

# Get Size of the OrderBook

Run some stats on the entire orderbook

In [41]:
contract = "XBTUSD"

book_size = orderbook_data(0, contract, raw_data=True)
book_size_avg = (book_size["total_buys"] + book_size["total_sells"]) // 2

curr_price = get_curr_price(contract)

ret = "Buyers" if book_size["total_buys"] > book_size["total_sells"] else "sellers"

if ret == "Buyers":
    amount = book_size["total_buys"] - book_size["total_sells"]
else:
    amount = book_size["total_sells"] - book_size["total_buys"]


print(f"\nCurrent {contract} price: ${curr_price:,.2f}")

print(f"\nThe average size of the order book for {contract} is: {book_size_avg}")
print(f"There are {amount} more orders from {ret}")

print(f'\nTotal orderbook size: {book_size["total_buys"]+book_size["total_sells"]}')
print(f'Total buys: {book_size["total_buys"]}')
print(f'Total sells: {book_size["total_sells"]}')

print(f"\nTotal buy orderbook volume: ${book_size['sum_of_buys']:,.2f}")
print(f"Total sell orderbook volume: ${book_size['sum_of_sells']:,.2f}")

print('\nFurthest 5 buy prices:')
for i, v in enumerate(book_size['furthest_5_buy_prices']):
    print(f"{i+1} => ${v:,.2f}")

print('\nFurthest 5 sell prices:')
for i, v in enumerate(book_size['furthest_5_sell_prices'][::-1]):
    print(f"{i+1} => ${v:,.2f}")


Current XBTUSD price: $19,233.00

The average size of the order book for XBTUSD is: 2630
There are 1383 more orders from sellers

Total orderbook size: 5261
Total buys: 1939
Total sells: 3322

Total buy orderbook volume: $78,154,800.00
Total sell orderbook volume: $107,601,300.00

Furthest 5 buy prices:
1 => $1,000,000.00
2 => $662,520.00
3 => $658,900.00
4 => $657,700.00
5 => $573,000.00

Furthest 5 sell prices:
1 => $0.50
2 => $1.00
3 => $1.50
4 => $10.00
5 => $30.00
